<a href="https://colab.research.google.com/github/thislis/XAI_Sentiment_Analysis/blob/main/R%26E_multi_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

github : https://github.com/thislis/XAI_Sentiment_Analysis/tree/main  
링크된 github에 데이터 출처 및 재가공에 쓰인 코드, 실험도구 정보와 실험중에 사용된 코드들이 첨부되어 있습니다.

In [ ]:
#필요한 도구들 다운로드

!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh
!pip install lime

In [ ]:
#구글 드라이브 마운트

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#필요 모듈 임포팅

#텍스트 전처리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#모델링
import re
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

#XAI_LIME
from lime.lime_text import LimeTextExplainer

In [ ]:
#데이터 로드

train1 = pd.read_csv('/content/drive/Shareddrives/R&E/dataset/train_anger.csv', names=['감정', '문장'], skiprows=1)
train2 = pd.read_csv('/content/drive/Shareddrives/R&E/dataset/train_anxiety.csv', names=['감정', '문장'], skiprows=1)
train3 = pd.read_csv('/content/drive/Shareddrives/R&E/dataset/train_embarrassment.csv', names=['감정', '문장'], skiprows=1)
train4 = pd.read_csv('/content/drive/Shareddrives/R&E/dataset/train_happy.csv', names=['감정', '문장'], skiprows=1)
train5 = pd.read_csv('/content/drive/Shareddrives/R&E/dataset/train_hurt.csv', names=['감정', '문장'], skiprows=1)
train6 = pd.read_csv('/content/drive/Shareddrives/R&E/dataset/train_sadness.csv', names=['감정', '문장'], skiprows=1)
train_data = pd.concat([train1, train2, train3, train4, train5, train6], ignore_index=True)
train_data.drop_duplicates(subset=['문장'], inplace=True) #중복 제거

test_data = pd.read_csv('/content/drive/Shareddrives/R&E/dataset/test_dataset.csv', names=['감정', '문장'], skiprows=1)
test_data.drop_duplicates(subset=['문장'], inplace=True) #중복 제거

In [ ]:
#정규 표현식 수행 및 공백을 Null값으로 변경

train_data['문장'] = train_data['문장'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data['문장'].replace('', np.nan, inplace=True)
test_data['문장'] = test_data['문장'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_data['문장'].replace('', np.nan, inplace=True)
tes_data = test_data.dropna(how='any')

In [ ]:
#불용어 정의
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '만', '되', '음', '면']

In [ ]:
#토큰화

mecab = Mecab() 

train_data['tokenized'] = train_data['문장'].apply(mecab.morphs)
train_data['tokenized'] = train_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])
test_data['tokenized'] = test_data['문장'].apply(mecab.morphs)
test_data['tokenized'] = test_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])

In [ ]:
#감정 정보를 0 또는 1로 변환(예 : 분노 모델 학습 시 분노를 1, 나머지를 0) - 함수 정의

def anger(text):
  if text == "분노":
    return 1
  else:
    return 0
    
def anxiety(text):
  if text == "불안":
    return 1
  else:
    return 0
  
def embarrassment(text):
  if text == "당황":
    return 1
  else:
    return 0

def happy(text):
  if text == "기쁨":
    return 1
  else:
    return 0

def hurt(text):
  if text == "상처":
    return 1
  else:
    return 0

def sadness(text):
  if text == "슬픔":
    return 1
  else:
    return 0

In [ ]:
#다중 감정 분석을 위하여 하나의 감정에 대한 이진 분류 모델을 여러개 생성 후 사용. 이를 위해 데이터 복사. 감정 정보를 0 또는 1로 변환(예 : 분노 모델 학습 시 분노를 1, 나머지를 0)

train_anger_data = train_data.copy()
train_anxiety_data = train_data.copy()
train_embarrassment_data = train_data.copy()
train_happy_data = train_data.copy()
train_hurt_data = train_data.copy()
train_sadness_data = train_data.copy()

test_anger_data = test_data.copy()
test_anxiety_data = test_data.copy()
test_embarrassment_data = test_data.copy()
test_happy_data = test_data.copy()
test_hurt_data = test_data.copy()
test_sadness_data = test_data.copy()

train_anger_data['감정'] = train_data['감정'].apply(anger)
test_anger_data['감정'] = test_data['감정'].apply(anger)
train_anxiety_data['감정'] = train_data['감정'].apply(anxiety)
test_anxiety_data['감정'] = test_data['감정'].apply(anxiety)
train_embarrassment_data['감정'] = train_data['감정'].apply(embarrassment)
test_embarrassment_data['감정'] = test_data['감정'].apply(embarrassment)
train_happy_data['감정'] = train_data['감정'].apply(happy)
test_happy_data['감정'] = test_data['감정'].apply(happy)
train_hurt_data['감정'] = train_data['감정'].apply(hurt)
test_hurt_data['감정'] = test_data['감정'].apply(hurt)
test_sadness_data['감정'] = train_data['감정'].apply(sadness)
train_sadness_data['감정'] = test_data['감정'].apply(sadness)

In [ ]:
#데이터의 토큰화된 문장과 감정을 X, Y로 분리

X_anger_train = train_anger_data['tokenized'].values
y_anger_train = train_anger_data['감정'].values
X_anger_test= test_anger_data['tokenized'].values
y_anger_test = test_anger_data['감정'].values

X_anxiety_train = train_anxiety_data['tokenized'].values
y_anxiety_train = train_anxiety_data['감정'].values
X_anxiety_test= test_anxiety_data['tokenized'].values
y_anxiety_test = test_anxiety_data['감정'].values

X_embarrassment_train = train_embarrassment_data['tokenized'].values
y_embarrassment_train = train_embarrassment_data['감정'].values
X_embarrassment_test= test_embarrassment_data['tokenized'].values
y_embarrassment_test = test_embarrassment_data['감정'].values

X_happy_train = train_happy_data['tokenized'].values
y_happy_train = train_happy_data['감정'].values
X_happy_test= test_happy_data['tokenized'].values
y_happy_test = test_happy_data['감정'].values

X_hurt_train = train_hurt_data['tokenized'].values
y_hurt_train = train_hurt_data['감정'].values
X_hurt_test= test_hurt_data['tokenized'].values
y_hurt_test = test_hurt_data['감정'].values

X_sadness_train = train_sadness_data['tokenized'].values
y_sadness_train = train_sadness_data['감정'].values
X_sadness_test= test_sadness_data['tokenized'].values
y_sadness_test = test_sadness_data['감정'].values

In [ ]:
#lstm의 사용을 위한 문장의 최대 길이 구하기

#def below_threshold_len(max_len, nested_list):
#  count = 0
#  for sentence in nested_list:
#    if(len(sentence) <= max_len):
#        count = count + 1
#  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))
#below_threshold_len(max_len, X_anger_train)

max_len = 35

In [ ]:
#분노 모델 학습

tokenizer_anger = Tokenizer()
tokenizer_anger.fit_on_texts(X_anger_train)

threshold_anger = 2
total_anger_cnt = len(tokenizer_anger.word_index) #단어 수
rare_anger_cnt = 0 #등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_anger_freq = 0 #훈련 데이터의 전체 단어 빈도수 총 합
rare_anger_freq = 0 #등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

#(단어, 빈도수)쌍->key와 value
for key, value in tokenizer_anger.word_counts.items():
    total_anger_freq = total_anger_freq + value
    if(value < threshold_anger):
        rare_anger_cnt = rare_anger_cnt + 1
        rare_anger_freq = rare_anger_freq + value

vocab_anger_size = total_anger_cnt - rare_anger_cnt + 2

tokenizer_anger = Tokenizer(vocab_anger_size, oov_token = 'OOV') 
tokenizer_anger.fit_on_texts(X_anger_train)
X_anger_train = tokenizer_anger.texts_to_sequences(X_anger_train)
X_anger_test = tokenizer_anger.texts_to_sequences(X_anger_test)

X_anger_train = pad_sequences(X_anger_train, maxlen=max_len)
X_anger_test = pad_sequences(X_anger_test, maxlen=max_len)

embedding_dim = 100
hidden_units = 128

model_anger = Sequential()
model_anger.add(Embedding(vocab_anger_size, embedding_dim))
model_anger.add(Bidirectional(LSTM(hidden_units))) # Bidirectional LSTM
model_anger.add(Dense(1, activation='sigmoid'))

es_anger = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc_anger = ModelCheckpoint('best_anger_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model_anger.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history_anger = model_anger.fit(X_anger_train, y_anger_train, epochs=15, callbacks=[es_anger, mc_anger], batch_size=128, validation_split=0.2)

In [ ]:
#불안 모델 학습

tokenizer_anxiety = Tokenizer()
tokenizer_anxiety.fit_on_texts(X_anxiety_train)

threshold_anxiety = 2
total_anxiety_cnt = len(tokenizer_anxiety.word_index) #단어 수
rare_anxiety_cnt = 0 #등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_anxiety_freq = 0 #훈련 데이터의 전체 단어 빈도수 총 합
rare_anxiety_freq = 0 #등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

#(단어, 빈도수)쌍->key와 value
for key, value in tokenizer_anxiety.word_counts.items():
    total_anxiety_freq = total_anxiety_freq + value
    if(value < threshold_anxiety):
        rare_anxiety_cnt = rare_anxiety_cnt + 1
        rare_anxiety_freq = rare_anxiety_freq + value

vocab_anxiety_size = total_anxiety_cnt - rare_anxiety_cnt + 2

tokenizer_anxiety = Tokenizer(vocab_anxiety_size, oov_token = 'OOV') 
tokenizer_anxiety.fit_on_texts(X_anxiety_train)
X_anxiety_train = tokenizer_anxiety.texts_to_sequences(X_anxiety_train)
X_anxiety_test = tokenizer_anxiety.texts_to_sequences(X_anxiety_test)

X_anxiety_train = pad_sequences(X_anxiety_train, maxlen=max_len)
X_anxiety_test = pad_sequences(X_anxiety_test, maxlen=max_len)

embedding_dim = 100
hidden_units = 128

model_anxiety = Sequential()
model_anxiety.add(Embedding(vocab_anxiety_size, embedding_dim))
model_anxiety.add(Bidirectional(LSTM(hidden_units))) # Bidirectional LSTM
model_anxiety.add(Dense(1, activation='sigmoid'))

es_anxiety = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc_anxiety = ModelCheckpoint('best_anxiety_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model_anxiety.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history_anxiety = model_anxiety.fit(X_anxiety_train, y_anxiety_train, epochs=15, callbacks=[es_anxiety, mc_anxiety], batch_size=128, validation_split=0.2)

In [ ]:
#당황 모델 학습

tokenizer_embarrassment = Tokenizer()
tokenizer_embarrassment.fit_on_texts(X_embarrassment_train)

threshold_embarrassment = 2
total_embarrassment_cnt = len(tokenizer_embarrassment.word_index) #단어 수
rare_embarrassment_cnt = 0 #등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_embarrassment_freq = 0 #훈련 데이터의 전체 단어 빈도수 총 합
rare_embarrassment_freq = 0 #등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

#(단어, 빈도수)쌍->key와 value
for key, value in tokenizer_embarrassment.word_counts.items():
    total_embarrassment_freq = total_embarrassment_freq + value
    if(value < threshold_embarrassment):
        rare_embarrassment_cnt = rare_embarrassment_cnt + 1
        rare_embarrassment_freq = rare_embarrassment_freq + value

vocab_embarrassment_size = total_embarrassment_cnt - rare_embarrassment_cnt + 2

tokenizer_embarrassment = Tokenizer(vocab_embarrassment_size, oov_token = 'OOV') 
tokenizer_embarrassment.fit_on_texts(X_embarrassment_train)
X_embarrassment_train = tokenizer_embarrassment.texts_to_sequences(X_embarrassment_train)
X_embarrassment_test = tokenizer_embarrassment.texts_to_sequences(X_embarrassment_test)

X_embarrassment_train = pad_sequences(X_embarrassment_train, maxlen=max_len)
X_embarrassment_test = pad_sequences(X_embarrassment_test, maxlen=max_len)

embedding_dim = 100
hidden_units = 128

model_embarrassment = Sequential()
model_embarrassment.add(Embedding(vocab_embarrassment_size, embedding_dim))
model_embarrassment.add(Bidirectional(LSTM(hidden_units))) # Bidirectional LSTM
model_embarrassment.add(Dense(1, activation='sigmoid'))

es_embarrassment = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc_embarrassment = ModelCheckpoint('best_embarrassment_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model_embarrassment.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history_embarrassment = model_embarrassment.fit(X_embarrassment_train, y_embarrassment_train, epochs=15, callbacks=[es_embarrassment, mc_embarrassment], batch_size=128, validation_split=0.2)

In [ ]:
#기쁨 모델 학습

tokenizer_happy = Tokenizer()
tokenizer_happy.fit_on_texts(X_happy_train)

threshold_happy = 2
total_happy_cnt = len(tokenizer_happy.word_index) #단어 수
rare_happy_cnt = 0 #등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_happy_freq = 0 #훈련 데이터의 전체 단어 빈도수 총 합
rare_happy_freq = 0 #등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

#(단어, 빈도수)쌍->key와 value
for key, value in tokenizer_happy.word_counts.items():
    total_happy_freq = total_happy_freq + value
    if(value < threshold_happy):
        rare_happy_cnt = rare_happy_cnt + 1
        rare_happy_freq = rare_happy_freq + value

vocab_happy_size = total_happy_cnt - rare_happy_cnt + 2

tokenizer_happy = Tokenizer(vocab_happy_size, oov_token = 'OOV') 
tokenizer_happy.fit_on_texts(X_happy_train)
X_happy_train = tokenizer_happy.texts_to_sequences(X_happy_train)
X_happy_test = tokenizer_happy.texts_to_sequences(X_happy_test)

X_happy_train = pad_sequences(X_happy_train, maxlen=max_len)
X_happy_test = pad_sequences(X_happy_test, maxlen=max_len)

embedding_dim = 100
hidden_units = 128

model_happy = Sequential()
model_happy.add(Embedding(vocab_happy_size, embedding_dim))
model_happy.add(Bidirectional(LSTM(hidden_units))) # Bidirectional LSTM
model_happy.add(Dense(1, activation='sigmoid'))

es_happy = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc_happy = ModelCheckpoint('best_happy_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model_happy.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history_happy = model_happy.fit(X_happy_train, y_happy_train, epochs=15, callbacks=[es_happy, mc_happy], batch_size=128, validation_split=0.2)

In [ ]:
#상처 모델 학습

tokenizer_hurt = Tokenizer()
tokenizer_hurt.fit_on_texts(X_hurt_train)

threshold_hurt = 2
total_hurt_cnt = len(tokenizer_hurt.word_index) #단어 수
rare_hurt_cnt = 0 #등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_hurt_freq = 0 #훈련 데이터의 전체 단어 빈도수 총 합
rare_hurt_freq = 0 #등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

#(단어, 빈도수)쌍->key와 value
for key, value in tokenizer_hurt.word_counts.items():
    total_hurt_freq = total_hurt_freq + value
    if(value < threshold_hurt):
        rare_hurt_cnt = rare_hurt_cnt + 1
        rare_hurt_freq = rare_hurt_freq + value

vocab_hurt_size = total_hurt_cnt - rare_hurt_cnt + 2

tokenizer_hurt = Tokenizer(vocab_hurt_size, oov_token = 'OOV') 
tokenizer_hurt.fit_on_texts(X_hurt_train)
X_hurt_train = tokenizer_hurt.texts_to_sequences(X_hurt_train)
X_hurt_test = tokenizer_hurt.texts_to_sequences(X_hurt_test)

X_hurt_train = pad_sequences(X_hurt_train, maxlen=max_len)
X_hurt_test = pad_sequences(X_hurt_test, maxlen=max_len)

embedding_dim = 100
hidden_units = 128

model_hurt = Sequential()
model_hurt.add(Embedding(vocab_hurt_size, embedding_dim))
model_hurt.add(Bidirectional(LSTM(hidden_units))) # Bidirectional LSTM
model_hurt.add(Dense(1, activation='sigmoid'))

es_hurt = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc_hurt = ModelCheckpoint('best_hurt_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model_hurt.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history_hurt = model_hurt.fit(X_hurt_train, y_hurt_train, epochs=15, callbacks=[es_hurt, mc_hurt], batch_size=128, validation_split=0.2)

In [ ]:
#슬픔 모델 학습

tokenizer_sadness = Tokenizer()
tokenizer_sadness.fit_on_texts(X_sadness_train)

threshold_sadness = 2
total_sadness_cnt = len(tokenizer_sadness.word_index) #단어 수
rare_sadness_cnt = 0 #등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_sadness_freq = 0 #훈련 데이터의 전체 단어 빈도수 총 합
rare_sadness_freq = 0 #등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

#(단어, 빈도수)쌍->key와 value
for key, value in tokenizer_sadness.word_counts.items():
    total_sadness_freq = total_sadness_freq + value
    if(value < threshold_sadness):
        rare_sadness_cnt = rare_sadness_cnt + 1
        rare_sadness_freq = rare_sadness_freq + value

vocab_sadness_size = total_sadness_cnt - rare_sadness_cnt + 2

tokenizer_sadness = Tokenizer(vocab_sadness_size, oov_token = 'OOV') 
tokenizer_sadness.fit_on_texts(X_sadness_train)
X_sadness_train = tokenizer_sadness.texts_to_sequences(X_sadness_train)
X_sadness_test = tokenizer_sadness.texts_to_sequences(X_sadness_test)

X_sadness_train = pad_sequences(X_sadness_train, maxlen=max_len)
X_sadness_test = pad_sequences(X_sadness_test, maxlen=max_len)

embedding_dim = 100
hidden_units = 128

model_sadness = Sequential()
model_sadness.add(Embedding(vocab_sadness_size, embedding_dim))
model_sadness.add(Bidirectional(LSTM(hidden_units))) # Bidirectional LSTM
model_sadness.add(Dense(1, activation='sigmoid'))

es_sadness = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc_sadness = ModelCheckpoint('best_sadness_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model_sadness.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history_sadness = model_sadness.fit(X_sadness_train, y_sadness_train, epochs=15, callbacks=[es_sadness, mc_sadness], batch_size=128, validation_split=0.2)

In [ ]:
#입력된 문장 전처리 함수 정의

def process_text(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = mecab.morphs(new_sentence) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  texts_clean = []
  for word in new_sentence:
    texts_clean.append(word)
  return " ".join(texts_clean)

In [ ]:
#분노 모델에 대한 설명자 정의

explainer_anger= LimeTextExplainer(class_names=['non anger', 'anger'])

def predict_anger_proba(arr):
  processed=[]
  for i in arr:
    processed.append(process_text(i))
  list_tokenized_ex = tokenizer_anger.texts_to_sequences(processed)
  Ex = pad_sequences(list_tokenized_ex, maxlen=max_len)
  pred=model_anger.predict(Ex)
  returnable=[]
  for i in pred:
    temp=i[0]
    returnable.append(np.array([1-temp,temp])) #I would recommend rounding temp and 1-temp off to 2 places
  return np.array(returnable)

In [ ]:
#불안 모델에 대한 설명자 정의

explainer_anxiety= LimeTextExplainer(class_names=['non anxiety', 'anxiety'])

def predict_anxiety_proba(arr):
  processed=[]
  for i in arr:
    processed.append(process_text(i))
  list_tokenized_ex = tokenizer_anxiety.texts_to_sequences(processed)
  Ex = pad_sequences(list_tokenized_ex, maxlen=max_len)
  pred=model_anxiety.predict(Ex)
  returnable=[]
  for i in pred:
    temp=i[0]
    returnable.append(np.array([1-temp,temp])) #I would recommend rounding temp and 1-temp off to 2 places
  return np.array(returnable)

In [ ]:
#당황 모델에 대한 설명자 정의

explainer_embarrassment= LimeTextExplainer(class_names=['non embarrassment', 'embarrassment'])

def predict_embarrassment_proba(arr):
  processed=[]
  for i in arr:
    processed.append(process_text(i))
  list_tokenized_ex = tokenizer_embarrassment.texts_to_sequences(processed)
  Ex = pad_sequences(list_tokenized_ex, maxlen=max_len)
  pred=model_embarrassment.predict(Ex)
  returnable=[]
  for i in pred:
    temp=i[0]
    returnable.append(np.array([1-temp,temp])) #I would recommend rounding temp and 1-temp off to 2 places
  return np.array(returnable)

In [ ]:
#기쁨 모델에 대한 설명자 정의

explainer_happy= LimeTextExplainer(class_names=['non happy', 'happy'])

def predict_happy_proba(arr):
  processed=[]
  for i in arr:
    processed.append(process_text(i))
  list_tokenized_ex = tokenizer_happy.texts_to_sequences(processed)
  Ex = pad_sequences(list_tokenized_ex, maxlen=max_len)
  pred=model_happy.predict(Ex)
  returnable=[]
  for i in pred:
    temp=i[0]
    returnable.append(np.array([1-temp,temp])) #I would recommend rounding temp and 1-temp off to 2 places
  return np.array(returnable)

In [ ]:
#상처 모델에 대한 설명자 정의

explainer_hurt= LimeTextExplainer(class_names=['non hurt', 'hurt'])

def predict_hurt_proba(arr):
  processed=[]
  for i in arr:
    processed.append(process_text(i))
  list_tokenized_ex = tokenizer_hurt.texts_to_sequences(processed)
  Ex = pad_sequences(list_tokenized_ex, maxlen=max_len)
  pred=model_hurt.predict(Ex)
  returnable=[]
  for i in pred:
    temp=i[0]
    returnable.append(np.array([1-temp,temp])) #I would recommend rounding temp and 1-temp off to 2 places
  return np.array(returnable)

In [ ]:
#슬픔 모델에 대한 설명자 정의

explainer_sadness= LimeTextExplainer(class_names=['non sadness', 'sadness'])

def predict_sadness_proba(arr):
  processed=[]
  for i in arr:
    processed.append(process_text(i))
  list_tokenized_ex = tokenizer_sadness.texts_to_sequences(processed)
  Ex = pad_sequences(list_tokenized_ex, maxlen=max_len)
  pred=model_sadness.predict(Ex)
  returnable=[]
  for i in pred:
    temp=i[0]
    returnable.append(np.array([1-temp,temp])) #I would recommend rounding temp and 1-temp off to 2 places
  return np.array(returnable)

In [ ]:
explainer_anger.explain_instance("안녕하세요!!",predict_anger_proba).show_in_notebook(text=True)

In [ ]:
explainer_anxiety.explain_instance("안녕하세요!!",predict_anxiety_proba).show_in_notebook(text=True)

In [ ]:
explainer_embarrassment.explain_instance("안녕하세요!!",predict_embarrassment_proba).show_in_notebook(text=True)

In [ ]:
explainer_happy.explain_instance("안녕하세요!!",predict_happy_proba).show_in_notebook(text=True)

In [ ]:
explainer_hurt.explain_instance("안녕하세요!!",predict_hurt_proba).show_in_notebook(text=True)

In [ ]:
explainer_sadness.explain_instance("안녕하세요!!",predict_sadness_proba).show_in_notebook(text=True)